In [ ]:
import requests

In [ ]:
topic = "Machine learning"

In [ ]:
import pandas as pd

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def search_wikipedia(topic):
    search_url = f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={topic}&format=json"
    response = requests.get(search_url)
    search_results = response.json()
    return search_results['query']['search']

def clean_snippet(snippet):
    soup = BeautifulSoup(snippet, "html.parser")
    return soup.get_text()

# Example usage
topic = "Machine learning"
search_results = search_wikipedia(topic)

# Extract titles from the search results
titles = [result['title'] for result in search_results]

print("Related topics:")
for title in titles:
    print(title)

# Optional: Convert the titles to a DataFrame if needed
df = pd.DataFrame(titles, columns=["Title"])


Related topics:
Machine learning
Quantum machine learning
Artificial intelligence in industry
Machine learning control
Neural network (machine learning)
Transformer (deep learning architecture)
Attention (machine learning)
Machine learning in bioinformatics
Boosting (machine learning)
Adversarial machine learning


In [ ]:
import requests

def get_wikipedia_extract(title, length="short"):
    # Define the endpoint
    endpoint = "https://en.wikipedia.org/w/api.php"

    # Set the extract length parameters
    if length == "short":
        exchars = 500  # Limit to approximately one paragraph
        params = {
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "exchars": exchars,
            "explaintext": True
        }
    elif length == "medium":
        exsentences = 5  # Limit to approximately two paragraphs
        params = {
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "exsentences": exsentences,
            "explaintext": True
        }
    else:
        params = {
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "exintro": True,
            "explaintext": True
        }

    # Make the request to the Wikipedia API
    response = requests.get(endpoint, params=params)
    data = response.json()

    # Extract the page information
    pages = data['query']['pages']
    page = next(iter(pages.values()))  # Get the first (and only) page

    # Extract the extract text
    extract = page.get("extract", "No extract found.")

    return extract

# Example usage
title = "Machine learning"
length = "full"  # Options: "short", "medium", "full"
extract = get_wikipedia_extract(title, length)
print(extract)


Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Recently, artificial neural networks have been able to surpass many previous approaches in performance.
ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.
The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning. 
From a theoretical viewpoint, probably

# OpenAI

In [33]:
from openai import OpenAI

client = OpenAI(
    api_key=''
)

In [34]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role':'system','content':'you are a good teacher who sets a good exam papers'},
        {'role': 'user','content':'create 50 multiple choice questions of topic machine learning with easy complexity, and mention all the answers at last.'}
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="**Machine Learning Multiple Choice Questions**\n\n1. What is machine learning?\n    A. A type of programming language\n    B. A branch of artificial intelligence\n    C. A type of software application\n    D. A physical machine that learns\n\n2. Which of the following is not a type of machine learning?\n    A. Supervised Learning\n    B. Unsupervised Learning\n    C. Reinforcement Learning\n    D. Manual Learning\n\n3. What is the goal of supervised learning?\n    A. To teach the model without any labels\n    B. To classify input data into predefined categories\n    C. To improve the performance of the model over time\n    D. To assess the model's decision-making abilities\n\n4. Which algorithm is commonly used for classification tasks in machine learning?\n    A. K-Means Clustering\n    B. Linear Regression\n    C. Support Vector Machines\n    D. Principal Component Analysis\n\n5. Which evaluation metric is commonly used for regression tasks?\n    A. Acc

# Manual Question generator

In [25]:
import re

def convert_to_question(sentence):
    # List of auxiliary verbs to identify questions
    question_words = ["is", "are", "am", "was", "were", "can", "could", "shall", "should", "will", "would", "do", "does", "did", "have", "has", "had", "may", "might", "must"]

    # Split the sentence into words
    words = sentence.split()

    # Regex to identify verbs
    verb_regex = re.compile(r"\bis\b|\bare\b|\bam\b|\bwas\b|\bwere\b|\bcan\b|\bcould\b|\bshall\b|\bshould\b|\bwill\b|\bwould\b|\bdo\b|\bdoes\b|\bdid\b|\bhave\b|\bhas\b|\bhad\b|\bmay\b|\bmight\b|\bmust\b")

    # Find the first auxiliary verb in the sentence
    for i, word in enumerate(words):
        if verb_regex.match(word.lower()):
            # Ensure we handle 'have' correctly
            if word.lower() == "have" and i != 0:
                question_sentence = "Do " + " ".join(words[:i] + [words[i]] + words[i+1:]) + "?"
            else:
                # Move the auxiliary verb to the front
                question_sentence = word + " " + " ".join(words[:i] + words[i+1:]) + "?"
            return question_sentence.capitalize()

    # Handle sentences without auxiliary verbs by adding 'do'
    if len(words) > 1:
        return "Do " + sentence + "?"
    else:
        return sentence + "?"

def paragraph_to_questions(paragraph):
    # Split the paragraph into sentences
    sentences = re.split(r'(?<=[.!?]) +', paragraph)
    questions = [convert_to_question(sentence.strip()) for sentence in sentences if sentence.strip()]
    return questions

# Example input paragraph
paragraph = """
This is not true
"""

# Generate questions
questions = paragraph_to_questions(paragraph)
for question in questions:
    print(question)


Is this not true?


# Huggin-Face model for generating question

In [ ]:
from transformers import pipeline

# Load the question generation pipeline
question_generator = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")


In [38]:
def convert_to_question(statement):
    # Add highlighting for the answer (this is required for the model to understand the context)
    input_text = "generate question: " + statement
    result = question_generator(input_text)
    return result[0]['generated_text']


# Convert a paragraph to questions for each line
def paragraph_to_questions(paragraph):
    lines = paragraph.split('\n')
    questions = [convert_to_question(line.strip()) for line in lines if line.strip()]
    return questions

In [44]:
# Example input paragraph
paragraph = """
Trip list includes names of Sejal and his boy-friend
"""

# Generate questions
questions = paragraph_to_questions(paragraph)
for question in questions:
    print(question)

What is Sejal's name on the trip list?
